In [8]:
import numpy as np
import pandas as pd
from my_stuff import MufexKeys
from quantfreedom.enums import *
from quantfreedom.exchanges.mufex_exchange.mufex import Mufex
from quantfreedom.indicators.tv_indicators import *
from quantfreedom.exchanges.exchange import Exchange
from quantfreedom.helper_funcs import dl_ex_candles, candles_to_df
np.set_printoptions(formatter={"float_kind": "{:0.1f}".format})
import numpy as np
import pandas as pd
import plotly.graph_objects as go
from plotly.subplots import make_subplots
from dash import Dash
from dash_bootstrap_templates import load_figure_template
from jupyter_dash import JupyterDash
from IPython import get_ipython
import dash_bootstrap_components as dbc

load_figure_template("darkly")
dbc_css = "https://cdn.jsdelivr.net/gh/AnnMarieW/dash-bootstrap-templates/dbc.min.css"
try:
    shell = str(get_ipython())
    if "ZMQInteractiveShell" in shell:
        app = JupyterDash(__name__, external_stylesheets=[dbc.themes.DARKLY, dbc_css])
    elif shell == "TerminalInteractiveShell":
        app = JupyterDash(__name__, external_stylesheets=[dbc.themes.DARKLY, dbc_css])
    else:
        app = Dash(__name__, external_stylesheets=[dbc.themes.DARKLY, dbc_css])
except NameError:
    app = Dash(__name__, external_stylesheets=[dbc.themes.DARKLY, dbc_css])

bg_color = "#0b0b18"

mufex_main = Mufex(use_test_net=False)

%load_ext autoreload
%autoreload 2

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


In [2]:
candles = mufex_main.get_candles(symbol="BTCUSDT", timeframe="5m", candles_to_dl=400)
open = candles[:1]
high = candles[:, 2]
low = candles[:, 3]
close = candles[:, 4]
close_shift = np.roll(candles[:, 4],1)

It took 00 mins and 01 seconds to download 400 candles


In [11]:
indicator = bb_sma_tv_calc(source=close, length=14, multi=2)

In [95]:
datetimes = pd.to_datetime(candles[:, 0], unit="ms")
bb_color = "48, 123, 255"
basic_color = "255, 176, 0"
fig = go.Figure(
    data=[
        go.Scatter(
            x=datetimes,
            y=indicator[:, 2],
            name="lower",
            line_color=f"rgb({bb_color})",
        ),
        go.Scatter(
            x=datetimes,
            y=indicator[:, 0],
            name="upper",
            line_color=f"rgb({bb_color})",
            fillcolor=f"rgba({bb_color}, 0.07)",
            fill="tonexty",
        ),
        go.Scatter(
            x=datetimes,
            y=indicator[:, 1],
            name="bb_sma",
            line_color=f"rgb({basic_color})",
        ),
        go.Candlestick(
            x=datetimes,
            open=candles[:, 1],
            high=candles[:, 2],
            low=candles[:, 3],
            close=candles[:, 4],
            name="Candles",
        ),
    ]
)
fig.update_layout(height=600, xaxis_rangeslider_visible=False)
fig.show()

In [58]:
from quantfreedom.plotting.plotting_base import plot_candles_1_ind_same_pane


plot_candles_1_ind_same_pane(candles=candles, indicator=indicator, ind_name=['upper', 'basis', 'lower'])